In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import display, Markdown

import pandas as pd
import numpy as np

import json

np.__version__

/var/folders/b7/c67wpzpn25q0h072y9mmz_th0000gn/T/ipykernel_74660/2341679603.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


'1.26.3'

# Pull in data

In [3]:
SOME_COMPLETE_DATSET = './data/22.2-OUTPUT-add-mjia-hand-labeling-and-relabel-uncategorized-using-them-as-exaples.csv'
df = pd.read_csv(SOME_COMPLETE_DATSET)
df

,performer,song,justification,love_song_sub_type
0,10cc,I'm Not In Love,"a repeated denial of being in love, which sugg...",It's Complicated
1,ABBA,Waterloo,"compares being in love with surrender, as if l...",Serenade
2,AWB,Cut The Cake,symbolically about marriage but focuses on the...,NaN
3,Aaliyah,Back & Forth,"a song about having a good time, but not a spe...",NaN
4,Aaron Neville,Everybody Plays The Fool,advising that everyone will be the fool in lov...,Longing & Heartbreak
...,...,...,...,...
5011,P!nk,Try,about the inevitability of getting hurt when l...,It's Complicated
5012,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",Give Me Everything,"about seizing the moment, with a blend of part...",Courtship & Anticipation
5013,Soko,We Might Be Dead By Tomorrow,emphasizing living in the current moment with ...,Longing & Heartbreak
5014,Taylor Swift,Blank Space,satirical view of the media's perception of he...,It's Complicated


# Split `performer` field into individual performers. We want a primary performer eg `Lady Gaga` and other performers `ft Justin Bieber` but as `Justin Bieber`.

### Open question: what's the max number of individual performers within the `performer` column? ie cuz if it's N, we can just have N columns, N-1 will be blank for most songs

In [4]:
import re

def split_on_regex(s, regex, index=0):
    parts = re.split(regex, s, flags=re.IGNORECASE)
    if parts.__len__() > 1 and index == 1:
        print([part.strip() for part in parts])
    if parts.__len__() > 3:
        raise ValueError(f'Too many performers in {s}')
    if index < len(parts):
        return parts[index].strip()
    return ''

split_df = df.copy()[['performer', 'song']]

split_df['primary_performer'] = split_df['performer'].apply(lambda x: split_on_regex(x, '(?i)featuring')).apply(lambda x: split_on_regex(x, '(?i)with')).apply(lambda x: split_on_regex(x, '(?i)&'))
split_df['secondary_performer'] = split_df['performer'].apply(lambda x: split_on_regex(x, '(?i)featuring', 1)).apply(lambda x: split_on_regex(x, '(?i)with', 1)).apply(lambda x: split_on_regex(x, '(?i)&', 1))
split_df['tertiary_performer'] = split_df['performer'].apply(lambda x: split_on_regex(x, '(?i)featuring', 2)).apply(lambda x: split_on_regex(x, '(?i)with', 2)).apply(lambda x: split_on_regex(x, '(?i)&', 2))

# Using pandas.py v1, concatenate all the split out performers into one giant list
pd.concat([split_df['primary_performer'], split_df['secondary_performer'], split_df['tertiary_performer']]).value_counts()

split_df

['Dawn', 'Tony Orlando']
['Erykah Badu', 'Common']
['Jennifer Lopez', 'Ja Rule']
["Lil' Flip", 'Lea']
['Paul Revere & The Raiders', 'Mark Lindsay']
['Plies', 'T-Pain']
['Rufus', 'Chaka Khan']
["Somethin' For The People", 'Trina & Tamara']
['The Partridge Family Starring Shirley Jones', 'David Cassidy']
['Truth Hurts', 'Rakim']
['Ciara', 'Petey Pablo']
['Gym Class Heroes', 'Patrick Stump']
['Ke$ha', '3OH!3']
['Lady Gaga', 'Beyonce']
['The Union Gap', 'Gary Puckett']
["'N Sync", 'Nelly']
['21 Savage & Metro Boomin', 'Drake']
['24kGoldn', 'iann dior']
['2Pac', 'K-Ci And JoJo']
['50 Cent', 'Justin Timberlake & Timbaland']
['50 Cent', 'Mobb Deep']
['50 Cent', 'Nate Dogg']
['50 Cent', 'Olivia']
['6ix9ine', 'Nicki Minaj & Murda Beatz']
['A$AP Rocky', 'Drake, 2 Chainz & Kendrick Lamar']
['Akon', 'Eminem']
['Akon', 'Snoop Dogg']
['Akon', 'Styles P.']
['Alicia Keys', 'Tony! Toni! Tone!']
['Allure', '112']
['Ariana Grande', 'Iggy Azalea']
['Ariana Grande', 'Mac Miller']
['Ariana Grande', 'Nicki M

,performer,song,primary_performer,secondary_performer,tertiary_performer
0,10cc,I'm Not In Love,10cc,,
1,ABBA,Waterloo,ABBA,,
2,AWB,Cut The Cake,AWB,,
3,Aaliyah,Back & Forth,Aaliyah,,
4,Aaron Neville,Everybody Plays The Fool,Aaron Neville,,
...,...,...,...,...,...
5011,P!nk,Try,P!nk,,
5012,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",Give Me Everything,Pitbull,,
5013,Soko,We Might Be Dead By Tomorrow,Soko,,
5014,Taylor Swift,Blank Space,Taylor Swift,,


In [5]:
# Print any songs containing the word " with ":
split_df[split_df['performer'].str.contains('(?i)/')]

,performer,song,primary_performer,secondary_performer,tertiary_performer
125,Olivia Newton-John/Electric Light Orchestra,Xanadu,Olivia Newton-John/Electric Light Orchestra,,
556,Barbra Streisand/Donna Summer,No More Tears (Enough Is Enough),Barbra Streisand/Donna Summer,,
908,Bryan Adams/Rod Stewart/Sting,All For Love,Bryan Adams/Rod Stewart/Sting,,
925,C+C Music Factory/F. Williams,Here We Go,C+C Music Factory/F. Williams,,
1326,Deliverance/Eric Weissberg & Steve Mandell,Dueling Banjos,Deliverance/Eric Weissberg,,
1911,George Michael/Elton John,Don't Let The Sun Go Down On Me,George Michael/Elton John,,
2374,Johnny Mathis/Deniece Williams,"Too Much, Too Little, Too Late",Johnny Mathis/Deniece Williams,,
2938,"Marvin Hamlisch/""The Sting""",The Entertainer,"Marvin Hamlisch/""The Sting""",,
3920,Stan Getz/Astrud Gilberto,The Girl From Ipanema,Stan Getz/Astrud Gilberto,,
4003,Sugarloaf/Jerry Corbetta,"Don't Call Us, We'll Call You",Sugarloaf/Jerry Corbetta,,


In [6]:
df['performer'].value_counts()

performer
Madonna                           36
Taylor Swift                      35
The Beatles                       33
Stevie Wonder                     25
Michael Jackson                   24
                                  ..
King Floyd                         1
Kim Wilde                          1
Brownstone                         1
Kid Rock Featuring Sheryl Crow     1
The Chainsmokers                   1
Name: count, Length: 2462, dtype: int64

In [7]:
# Demo of the array spread operator spreading one array into another:
[1, 2, 3, *['a', 'b', 'c']]


[1, 2, 3, 'a', 'b', 'c']

In [8]:
GPT_SPLIT_PROMPT = """
You are an expert pop music critic, familiar with all the musical artists of the last 70 years.

Given a string containing the performer or performers for a song, return a list of the performers in the same order they appear in the string.

Sample input:
```json
[
    "Kid Rock Featuring Sheryl Crow", 
    "Pitbull Featuring Ne-Yo, Afrojack & Nayer", 
    "The Partridge Family Starring Shirley Jones Featuring David Cassidy", 
    "FEFE ft. Nicki Minaj, Murda Beatz",
    "The Beatles With Billy Preston",
    "Johnny Mathis/Deniece Williams"
]
```

Sample output:
```json
{"output":
[
    ["Kid Rock", "Sheryl Crow"],
    ["Pitbull", "Ne-Yo", "Afrojack", "Nayer"],
    ["The Partridge Family", "Shirley Jones", "David Cassidy"],
    ["FEFE", "Nicki Minaj", "Murda Beatz"],
    ["The Beatles", "Billy Preston"],
    ["Johnny Mathis", "Deniece Williams"]
]
}
```

Ok, now do this next input:
"""

# JUST USE GPT... sigh

# Fetch new labels using GPT4

In [9]:
import os
from openai import OpenAI

# First, activate the './.env' file which sets the OPENAI_API_KEY environment variable using python-dotenv
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [10]:
import ast

def eval_this_str(s):
    return ast.literal_eval(s)

In [11]:
def get_love_song_labels_for_rows(row_subset_str):
    try: 
        completion = client.chat.completions.create(
          model="gpt-4-1106-preview",
          response_format={'type': "json_object"},
          messages=[
            {"role": "user", "content": GPT_SPLIT_PROMPT},
            {"role": "user", "content": row_subset_str},
          ]
        )
        content = completion.choices[0].message.content
        # return content
        response_content = eval_this_str(content)['output']
        # print(f"Response contains {completion['usage']['completion_tokens']} tokens out of 4096 maximum allowed tokens.")
        return response_content
    except Exception as e:
        print('ERROR FETCHING', e)
        return []

### Fetch in batches

In [47]:
NUM_TOTAL_COLUMNS = 15 # play it safe
def fetch_all_data_in_batches(df, increment=25):
    gpt_labeled_result = []
    for i in range(101):
        subset = df['performer'][i*increment : (i+1)*increment].reset_index(drop=True)
        if len(subset):
            subset_as_json_string = subset.to_json(orient='values')
            print([i*increment, (i+1)*increment])
            print(subset_as_json_string, '\n')
            fetched_and_formatted_res = get_love_song_labels_for_rows(subset_as_json_string)
            # fetched_and_formatted_res = [['The DeFranco Family', 'Tony DeFranco']]
            # Append the original performer string as the first element in each sub array (that's effectively the id so we can merge it in later)
            fetched_and_formatted_res = [[subset[j], *fetched_and_formatted_res[j]] for j in range(len(fetched_and_formatted_res))]
            # Ok, we need all the sub arrays to have the same number of elements, so we'll pad the shorter ones with empty strings
            for j in range(len(fetched_and_formatted_res)):
                fetched_and_formatted_res[j] = fetched_and_formatted_res[j] + ['']*(NUM_TOTAL_COLUMNS - len(fetched_and_formatted_res[j]))
            print(fetched_and_formatted_res)
            gpt_labeled_result = [*gpt_labeled_result, *fetched_and_formatted_res]
    return gpt_labeled_result

sample_result = fetch_all_data_in_batches(df[df.performer.str.contains(', ')].head(5))

sample_df = pd.DataFrame(
        sample_result, 
        # playing it safe LOL
        columns=['performer', *[f'performer_{i}' for i in range(1, NUM_TOTAL_COLUMNS)]]
)    
sample_df

[0, 25]
["Zac Efron, Andrew Seeley & Vanessa Anne Hudgens","A$AP Rocky Featuring Drake, 2 Chainz & Kendrick Lamar","Babyface Featuring LL Cool J, Howard Hewett, Jody Watley & Jeffrey Daniels","Barry DeVorzon and Perry Botkin, Jr.","Blood, Sweat & Tears"] 

[['Zac Efron, Andrew Seeley & Vanessa Anne Hudgens', 'Zac Efron', 'Andrew Seeley', 'Vanessa Anne Hudgens', '', '', '', '', '', '', '', '', '', '', ''], ['A$AP Rocky Featuring Drake, 2 Chainz & Kendrick Lamar', 'A$AP Rocky', 'Drake', '2 Chainz', 'Kendrick Lamar', '', '', '', '', '', '', '', '', '', ''], ['Babyface Featuring LL Cool J, Howard Hewett, Jody Watley & Jeffrey Daniels', 'Babyface', 'LL Cool J', 'Howard Hewett', 'Jody Watley', 'Jeffrey Daniels', '', '', '', '', '', '', '', '', ''], ['Barry DeVorzon and Perry Botkin, Jr.', 'Barry DeVorzon', 'Perry Botkin, Jr.', '', '', '', '', '', '', '', '', '', '', '', ''], ['Blood, Sweat & Tears', 'Blood, Sweat & Tears', '', '', '', '', '', '', '', '', '', '', '', '', '']]


,performer,performer_1,performer_2,performer_3,performer_4,performer_5,performer_6,performer_7,performer_8,performer_9,performer_10,performer_11,performer_12,performer_13,performer_14
0,"Zac Efron, Andrew Seeley & Vanessa Anne Hudgens",Zac Efron,Andrew Seeley,Vanessa Anne Hudgens,,,,,,,,,,,
1,"A$AP Rocky Featuring Drake, 2 Chainz & Kendric...",A$AP Rocky,Drake,2 Chainz,Kendrick Lamar,,,,,,,,,,
2,"Babyface Featuring LL Cool J, Howard Hewett, J...",Babyface,LL Cool J,Howard Hewett,Jody Watley,Jeffrey Daniels,,,,,,,,,
3,"Barry DeVorzon and Perry Botkin, Jr.",Barry DeVorzon,"Perry Botkin, Jr.",,,,,,,,,,,,
4,"Blood, Sweat & Tears","Blood, Sweat & Tears",,,,,,,,,,,,,


In [52]:
def get_file_path(postfix):
    return f'./data/22.3-TEMP-gpt4-performer-split-{postfix}.csv'

In [53]:
sample_df.to_csv(get_file_path('TEST'), index=False)

In [56]:
def save_to_temp_csv(formatted_gpt_result, run_num):
    first_label_pass_df = pd.DataFrame(
        formatted_gpt_result, 
        # playing it safe LOL
        columns=['performer', *[f'performer_{i}' for i in range(1, NUM_TOTAL_COLUMNS)]]
    )
    first_label_pass_df.to_csv(get_file_path(run_num), index=False)

# (run a test call to be sure things look ok)
For 100 rows at once:
For GPT4 turbo: 3min, about 3400 tokens out of 4096 maximum allowed tokens 

In [57]:
TEST_POSTFIX = 'JUST A TEST, DELETE ME'
save_to_temp_csv(
    fetch_all_data_in_batches(
        df[df.performer.str.contains(', ')].head(5)
    ),
    TEST_POSTFIX
)

[0, 25]
["Zac Efron, Andrew Seeley & Vanessa Anne Hudgens","A$AP Rocky Featuring Drake, 2 Chainz & Kendrick Lamar","Babyface Featuring LL Cool J, Howard Hewett, Jody Watley & Jeffrey Daniels","Barry DeVorzon and Perry Botkin, Jr.","Blood, Sweat & Tears"] 

[['Zac Efron, Andrew Seeley & Vanessa Anne Hudgens', 'Zac Efron', 'Andrew Seeley', 'Vanessa Anne Hudgens', '', '', '', '', '', '', '', '', '', '', ''], ['A$AP Rocky Featuring Drake, 2 Chainz & Kendrick Lamar', 'A$AP Rocky', 'Drake', '2 Chainz', 'Kendrick Lamar', '', '', '', '', '', '', '', '', '', ''], ['Babyface Featuring LL Cool J, Howard Hewett, Jody Watley & Jeffrey Daniels', 'Babyface', 'LL Cool J', 'Howard Hewett', 'Jody Watley', 'Jeffrey Daniels', '', '', '', '', '', '', '', '', ''], ['Barry DeVorzon and Perry Botkin, Jr.', 'Barry DeVorzon', 'Perry Botkin, Jr.', '', '', '', '', '', '', '', '', '', '', '', ''], ['Blood, Sweat & Tears', 'Blood, Sweat & Tears', '', '', '', '', '', '', '', '', '', '', '', '', '']]


In [59]:
pd.read_csv(get_file_path(TEST_POSTFIX))

,performer,performer_1,performer_2,performer_3,performer_4,performer_5,performer_6,performer_7,performer_8,performer_9,performer_10,performer_11,performer_12,performer_13,performer_14
0,"Zac Efron, Andrew Seeley & Vanessa Anne Hudgens",Zac Efron,Andrew Seeley,Vanessa Anne Hudgens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"A$AP Rocky Featuring Drake, 2 Chainz & Kendric...",A$AP Rocky,Drake,2 Chainz,Kendrick Lamar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Babyface Featuring LL Cool J, Howard Hewett, J...",Babyface,LL Cool J,Howard Hewett,Jody Watley,Jeffrey Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Barry DeVorzon and Perry Botkin, Jr.",Barry DeVorzon,"Perry Botkin, Jr.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Blood, Sweat & Tears","Blood, Sweat & Tears",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
